<a href="https://colab.research.google.com/github/RamyaMangu/RamyaMangu/blob/main/Bioannotator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**LIBRARIES**

In [87]:
import urllib.request, urllib.error, urllib.parse
import json
import os
import pandas as pd
from pathlib import Path
from pprint import pprint
import string
import re
from tabulate import tabulate
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import plotly.offline as pyo

**DATA IMPORT**

In [2]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/Shareddrives/ASR Recording Modality Errors Analysis/Data/ASR transcribed data
dirName_a1 = '/content/drive/Shareddrives/ASR Recording Modality Errors Analysis/Data/ASR transcribed data/Amazon Transcribe Medical - 2022-03-03/CSV for Evaluation'
data_files_a1 = list() #medical amazon folder
for root, dirs, files in os.walk(dirName_a1):
    for file in files:
            data_files_a1 += [os.path.join(dirName_a1, file)]

dirName_a2 = '/content/drive/Shareddrives/ASR Recording Modality Errors Analysis/Data/ASR transcribed data/Amazon Transcribe General - 2022-03-03/CSV for Evaluation'
data_files_a2 = list() #general amazon folder
for root, dirs, files in os.walk(dirName_a2):
    for file in files:
            data_files_a2 += [os.path.join(dirName_a2, file)]

dirName_g1 = '/content/drive/Shareddrives/ASR Recording Modality Errors Analysis/Data/ASR transcribed data/Google Speech to Text medical_conversations - 2022-03-03/CSV for evaluation'
data_files_g1 = list() #medical amazon folder
for root, dirs, files in os.walk(dirName_g1):
    for file in files:
            data_files_g1 += [os.path.join(dirName_g1, file)]

dirName_g2 = '/content/drive/Shareddrives/ASR Recording Modality Errors Analysis/Data/ASR transcribed data/Google Speech to Text Video - 2022-03-03/CSV for Evaluation'
data_files_g2 = list() #medical amazon folder
for root, dirs, files in os.walk(dirName_g2):
    for file in files:
            data_files_g2 += [os.path.join(dirName_g2, file)]
#So now we have the list of files for each engine

# Now for the reference directory
%cd /content/drive/Shareddrives/ASR Recording Modality Errors Analysis/Data/Original conversation Data/cleaned for asr eval/cleaned csv reader scripts for asr eval
dirName_ref = '/content/drive/Shareddrives/ASR Recording Modality Errors Analysis/Data/Original conversation Data/cleaned for asr eval/cleaned csv reader scripts for asr eval'
datafiles_ref = list()
for root, dirs, files in os.walk(dirName_ref):
    for file in files:
            datafiles_ref += [os.path.join(dirName_ref, file)]


Mounted at /content/drive
/content/drive/Shareddrives/ASR Recording Modality Errors Analysis/Data/ASR transcribed data
/content/drive/Shareddrives/ASR Recording Modality Errors Analysis/Data/Original conversation Data/cleaned for asr eval/cleaned csv reader scripts for asr eval


**Removing punctuations and Converting each file into string of words**

In [3]:
def convert(data_f, dict_df):
  length = len(data_f)
  for i in range(length):
    tf = pd.read_csv(data_f[i])
    string_trans = ' '.join(tf['ASR Text'].to_list())
    translator=str.maketrans('','',string.punctuation) #translator to remove the punctuations
    string_trans=string_trans.translate(translator)
    id = tf['Hyp Visit ID'].iloc[0]
    dict_df[id] = string_trans

def ref_convert(data_f, dict_df):
  for i in range(len(data_f)):
    tf = pd.read_csv(data_f[i])
    string_trans = ' '.join(tf['Ref Word'].to_list())
    id = tf['Ref Visit ID'].iloc[0]
    dict_df[id] = string_trans

In [4]:
a1 = {}
convert(data_files_a1, a1)
a2 = {}
convert(data_files_a2, a2)
g1 = {}
convert(data_files_g1, g1)
g2 = {}
convert(data_files_g2, g2)
#so now a1, a2, g1 aand g2 are our dictionaries that have the visit id's as keys and strings of words as values from each transcipt without punctuations.
ref = {}
ref_convert(datafiles_ref, ref)

**Bioannotator API import**

In [5]:
# getting our bioannotator json file
REST_URL = "http://data.bioontology.org"
API_KEY = ""


def get_json(url):
    opener = urllib.request.build_opener()
    opener.addheaders = [('Authorization', 'apikey token=' + API_KEY)]
    return json.loads(opener.open(url).read())


def print_annotations(annotations, get_class=True):
    for result in annotations:
        class_details = result["annotatedClass"]
        if get_class:
            try:
                class_details = get_json(result["annotatedClass"]["links"]["self"])
            except urllib.error.HTTPError:
                print(f"Error retrieving {result['annotatedClass']['@id']}")
                continue
        print("Class details")
        print("\tid: " + class_details["@id"])
        print("\tprefLabel: " + class_details["prefLabel"])
        print("\tontology: " + class_details["links"]["ontology"])

        print("Annotation details")
        for annotation in result["annotations"]:
            print("\tfrom: " + str(annotation["from"]))
            print("\tto: " + str(annotation["to"]))
            print("\tmatch type: " + annotation["matchType"])

        if result["hierarchy"]:
            print("\n\tHierarchy annotations")
            for annotation in result["hierarchy"]:
                try:
                    class_details = get_json(annotation["annotatedClass"]["links"]["self"])
                except urllib.error.HTTPError:
                    print(f"Error retrieving {annotation['annotatedClass']['@id']}")
                    continue
                pref_label = class_details["prefLabel"] or "no label"
                print("\t\tClass details")
                print("\t\t\tid: " + class_details["@id"])
                print("\t\t\tprefLabel: " + class_details["prefLabel"])
                print("\t\t\tontology: " + class_details["links"]["ontology"])
                print("\t\t\tdistance from originally annotated class: " + str(annotation["distance"]))

        print("\n\n")

**Chunking the files and sending it to the bioannotator**

In [6]:
def chunk(lis, an, an2):
  length_lis = len(lis)
  l =0
  for key, value in lis.items(): # iterating through the dictionary
    ind_len = divmod(len(value), 15) #splitting the string into 15 parts
    j = 0
    string_ch = ""
    str_ch =[]
    for h in value:
      string_ch += h
      if j == ind_len[0]:
        str_ch.append(string_ch)
        string_ch =""
        j = 0
      else:
        j= j +1
    an[key] = {} # creating a dictionary for each visit id that holds the annotation ids as keys and its occurences as values like [visitId][annId][no.of occurences]
    an2[key] = list() # this will used for the json file
    for k in range(len(str_ch)):
      annotations = get_json(REST_URL + "/annotator?text=" + urllib.parse.quote(str_ch[k]))
      an2[key].extend(annotations) # adding the complete annotations to the dictionary
      for l in range(len(annotations)):
        val = annotations[l]['annotatedClass']['@id']
        if val in an[key]:
          an[key][val]+=1
        else:
          an[key][val]= 1

In [7]:
#chunking all the files and sending it to the bioannotator
anid_ama_med ={}
an_amed = {}
anid_ama_gen ={}
an_agen = {}
anid_gog_med={}
an_gmed = {}
anid_gog_gen={} 
an_ggen = {}
anid_ref={}
an_ref = {}
chunk(a1, anid_ama_med, an_amed)

In [8]:
chunk(a2, anid_ama_gen, an_agen)

In [9]:
chunk(g1, anid_gog_med, an_gmed)


In [10]:
chunk(g2, anid_gog_gen, an_ggen)

In [11]:
chunk(ref, anid_ref, an_ref)

In [12]:
anids = {"Amazon Transcribe Medical":anid_ama_med, 
         "Amazon Transcribe General":anid_ama_gen, 
         "Google Speech-to-Text Medical_Conversations":anid_gog_med, 
         "Google Speech-to-Text Video":anid_gog_gen}
ans = {"Amazon Transcribe Medical":[an_amed, a1], 
         "Amazon Transcribe General":[an_agen, a2], 
         "Google Speech-to-Text Medical_Conversations":[an_gmed, g1], 
         "Google Speech-to-Text Video":[an_ggen, g2]}

**Calculating Precision and Recall**

In [13]:
# We calculate the true positives, false positives and false negatives through the below function and then calculate the precision and recall manually
#true positive is when the annotation label is there in both reference and the transcribed file
#false negative is when the annotation label is there in reference but not in the transcribed file
#false positive is when the annotation label is there in the transcibed files but not in the reference
def calculate_tp_fn(ref, trans):
  tp =0
  fn =0
  for key in ref:
    if key in trans:
      tp+= 1
    fn+=1
  return(tp,fn)

def calculate_fp(ref, trans):
  fp = 0
  for key in trans:
    if key not in ref:
      fp+=1
  return fp

**Creating an output table for each Visit ID**

In [14]:
columns= [
          'Visit ID', 'Engine', 'True Positive', 'False Positive', 'Precision', 'Recall', 'F1 Score' 
]

In [24]:
path = "/content/drive/Shareddrives/ASR Recording Modality Errors Analysis/Data/Bioannotator_Analysis_ASR_transcribed data/CSV"
def create_output(ref, dict_ids, values, vi_metrics):
  for key, value in ref.items():
    data = pd.DataFrame(columns=columns)
    vi_metrics[key] = {} # dict for each visit id
    # iterating through all the four folders
    for k,val in dict_ids.items():
      ids = val[key] #taking the values of that particular id
      tp, fn = calculate_tp_fn(value, ids)
      # we will storing the values with their respective engine names like so [engine type]{tp, fp, fn, recall...} and we will be updating the values accordingly
      if k not in values:
        values[k] ={}
        values[k]['tp'] =0
        values[k]['fp'] =0 
        values[k]['fn'] =0 
        values[k]['recall'] =0 
        values[k]['precision'] =0 
        values[k]['f1'] =0 
      values[k]['tp'] += tp
      values[k]['fn'] += fn
      recall = tp/(tp+fn)
      values[k]['recall'] += recall
      fp = calculate_fp(value, ids)
      values[k]['fp'] += fp
      precision = tp/(tp+fp)
      values[k]['precision'] += precision
      f1_score = (2 * precision * recall)/(precision + recall)
      values[k]['f1'] += f1_score
      vi_metrics[key][k] = list()
      vi_metrics[key][k].extend([tp])
      vi_metrics[key][k].extend([fn])
      vi_metrics[key][k].extend([fp])
      vi_metrics[key][k].extend([recall])
      vi_metrics[key][k].extend([precision])
      vi_metrics[key][k].extend([f1_score])
      #data.loc[len(data.index)]= [key,k,tp,fp,round(precision, 2),round(recall,2),round(f1_score, 2)]
     #filePathNameWExt =  path + '/' + str(key) + '_Bioannotator_anlaysis' + '.csv'
   # data.to_csv(filePathNameWExt)


In [26]:
values = {} # this will be storing the sum of the values of the tps, fps, fns, recall, precision to help us compute the macro and micro average later.
vi_metrics = {} # dict that stores the fp, tp, fn, recall and precision by visit ID. In the order tp, fp, fn, recall and f1 score - format [visitId][Engine] = [tp, fn, fp, recall, prec]
create_output(anid_ref, anids, values, vi_metrics)

**Create a JSON file**

In [ ]:
def create_json(path, annot):
  for result in annot:
    class_details = result["annotatedClass"]
    class_details = get_json(result["annotatedClass"]["links"]["self"])
    newdat = {
      'Class Details' : [
                         {
                             'id:' : class_details["@id"],
                             'prefLabel:' : class_details["prefLabel"],
                             'ontology:' : class_details["links"]["ontology"]
                         }
      ]
   }
    path.append(newdat)
    for annotation in result["annotations"]:
      annotdat={
          'Annotation Details' : [{
              "from: " : str(annotation["from"]),
              "to: " : str(annotation["to"]),
              "match type: " : annotation["matchType"]
          }]
    }
      path.append(annotdat)
    #j_string = json.dumps(newdat)

In [ ]:
ppath = "/content/drive/Shareddrives/ASR Recording Modality Errors Analysis/Data/Bioannotator_Analysis_ASR_transcribed data/JSON"
for key, value in an_ref.items():
  direc = str(key)
  path = os.path.join(ppath, direc)
  if not os.path.isdir(path):
    os.makedirs(path)
  dat = [{
      'Reference' : [{'Visit Id': int(key)}, {'String' :ref[int(key)]}]
      
  }]
  filejson = path + '/' + str(key) + '_Reference_Bioannotator_anlaysis' + '.json'
  #j_string = json.dumps(dat)
  create_json(dat, value)
  with open(filejson, 'w') as j_file:
      json.dump(dat, j_file)
  for k, v1 in ans.items():
    filejson = path + '/' + str(key) + '_'+ k +'_Bioannotator_anlaysis' + '.json'
    newdat = [{
      k : {'Visit Id': int(key)},
      'Sent in string' : v1[1][key]
    }]
    create_json(newdat, v1[0][key])
    with open(filejson, 'w') as j_file:
      json.dump(newdat, j_file)

**Creating The Output Table**

In [27]:
head = ["Engine Type", "Recall Macro Average", "Recall Micro Average", "Precision Macro Average", "Precision Micro Average", "F1 Macro Average", "F1 Micro Average"]
data = list()
for k, val in values.items():
  micRec = val['tp'] / (val['tp'] + val['fn'])
  micPrec = val['tp'] / (val['tp'] + val['fp'])
  micf1 = (2 * micRec * micPrec)/(micPrec+micRec)
  macf1 = val['f1']/len(ref)
  macrec = val['recall']/len(ref)
  macprec = val['precision']/len(ref)
  ls = [k, macrec, micRec, macprec, micPrec, macf1, micf1]
  data.append(ls)

print(tabulate(data, headers=head, tablefmt='fancy_grid'))

╒═════════════════════════════════════════════╤════════════════════════╤════════════════════════╤═══════════════════════════╤═══════════════════════════╤════════════════════╤════════════════════╕
│ Engine Type                                 │   Recall Macro Average │   Recall Micro Average │   Precision Macro Average │   Precision Micro Average │   F1 Macro Average │   F1 Micro Average │
╞═════════════════════════════════════════════╪════════════════════════╪════════════════════════╪═══════════════════════════╪═══════════════════════════╪════════════════════╪════════════════════╡
│ Amazon Transcribe Medical                   │               0.479348 │               0.479991 │                  0.945213 │                  0.943825 │           0.636024 │           0.636357 │
├─────────────────────────────────────────────┼────────────────────────┼────────────────────────┼───────────────────────────┼───────────────────────────┼────────────────────┼────────────────────┤
│ Amazon Transcribe 

**Analysis of Analysis** 

- **Bar graphs for tp, fp and fn for each visit id**

In [84]:
def plot(labels, metric, dict_eng):
  x = np.arange(40)
  lab = []
  for k, val in dict_eng.items():
    lab.append(k)
  width = 0.3
  plt.figure(figsize=(40, 10))
  plt.bar(x-0.6, dict_eng[lab[0]], width, color='cyan')
  plt.bar(x-0.3, dict_eng[lab[1]], width, color='orange')
  plt.bar(x, dict_eng[lab[2]], width, color='green')
  plt.bar(x+0.3, dict_eng[lab[3]], width, color='blue')
  plt.xticks(x, labels)
  plt.xlabel("Visit Ids")
  plt.ylabel(metric)
  plt.legend(lab)
  
def bar_graph(dict_vi):
  lab = ['tp', 'fp', 'fn', 'recall', 'precision', 'f1 score']
  for i in range(0, 7):
    labels = [] # This will store the visit ids
    eng_metri = dict()
    for key, value in dict_vi.items():
      labels.append(key)
      for val in value:
        eng_metri[val] = list()
        eng_metri[val].extend([value[val][i]])
    
    plot(labels, lab[i], eng_metri)
    plt.show()




- **Radar graph based on each engine**

In [94]:
categories = ["Amazon Transcribe Medical", "Amazon Transcribe General", "Google Speech-to-Text Medical_Conversations", "Google Speech-to-Text Video"]
recall =[]
precision =[]
f1 =[]
for k, val in values.items():
  macf1 = val['f1']/len(ref)
  macrec = val['recall']/len(ref)
  macprec = val['precision']/len(ref)
  recall.append(macrec)
  precision.append(macprec)
  f1.append(macf1)

fig = go.Figure(
    data=[
        go.Scatterpolar(r=recall, theta=categories, name='Recall'),
        go.Scatterpolar(r=precision, theta=categories, name='Precision'),
        go.Scatterpolar(r=f1, theta=categories, name='F1 Score')
    ],
    layout=go.Layout(
        title=go.layout.Title(text='Performance'),
        polar={'radialaxis': {'visible': True}},
        showlegend=True
    )
)

pyo.plot(fig)
fig.show()